In [2]:
!python -V

Python 3.10.13


In [2]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [4]:
df1 = pd.read_parquet('/workspaces/mlopszoomcamp24/data/yellow_tripdata_2023-01.parquet')
# Convert all column names to lowercase
df1.columns = [col.lower() for col in df1.columns]

df2 = pd.read_parquet('/workspaces/mlopszoomcamp24/data/yellow_tripdata_2023-02.parquet')
df2.columns = [col.lower() for col in df2.columns]

In [5]:
df1.head(2)

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0


In [6]:
#df = pd.concat([df1, df2], axis=0)

# Reset indexes of original DataFrames
df1.reset_index(drop=True, inplace=True)
df2.reset_index(drop=True, inplace=True)

# Concatenate the DataFrames
df = pd.concat([df1, df2], ignore_index=True)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5980721 entries, 0 to 5980720
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   vendorid               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   ratecodeid             float64       
 6   store_and_fwd_flag     object        
 7   pulocationid           int64         
 8   dolocationid           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

### Homework1

#### Q1. Downloading the data

In [8]:
# Count the number of columns in the DataFrame
num_columns = df1.shape[1]

# Print the number of columns
print("Number of columns:", num_columns)


Number of columns: 19


#### Q2. Computing duration

In [9]:
# Calculate the duration of each trip in minutes and store it in a new column 'duration'
# The duration is calculated as the difference between drop-off and pickup times
df1['duration'] = df1.tpep_dropoff_datetime - df1.tpep_pickup_datetime
# Convert the duration from timedelta to minutes by dividing total seconds by 60
df1.duration = df1.duration.apply(lambda td: td.total_seconds() / 60)

# Calculate the standard deviation of the 'duration' column
duration_std1 = df1['duration'].std()

# Print the standard deviation
print("Standard deviation of duration column:", duration_std1)


Standard deviation of duration column: 42.59435124195458


#### Q3. Dropping outliers

In [10]:
# Count the total number of records before removing outliers
total_records_before = len(df1)

# Filter the DataFrame to keep only records where the duration is between 1 and 60 minutes
df1_filtered = df1[(df1['duration'] >= 1) & (df1['duration'] <= 60)]

# Count the number of records after removing outliers
total_records_after = len(df1_filtered)

# Calculate the fraction of records remaining after dropping outliers
fraction_remaining = total_records_after / total_records_before *100

# Print the fraction of records remaining
print("Fraction of records remaining after dropping outliers:", fraction_remaining, "%")


Fraction of records remaining after dropping outliers: 98.1220282212598 %


##### Q4. One-hot encoding

In [11]:


# Convert DataFrame to a list of dictionaries
data_list = df1_filtered[['pulocationid', 'dolocationid']].astype(str).to_dict(orient='records')


#data_list

# Initialize and fit the dictionary vectorizer
dv  = DictVectorizer(sparse=True)
#dv.fit(data_list)

# Transform the data to obtain the feature matrix
feature_matrix = dv.fit_transform(data_list)
train_dicts = dv.fit_transform(data_list)
# Get the dimensionality of the feature matrix (number of columns)
num_columns = feature_matrix.shape[1]

# Print the dimensionality of the feature matrix
print("Dimensionality of the feature matrix (number of columns):", num_columns)

Dimensionality of the feature matrix (number of columns): 515


In [12]:
#sns.distplot(y_pred, label='prediction')
#sns.distplot(y_train, label='actual')

#plt.legend()

##### Q 5 and 6. Evaluating the model

In [3]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df.columns = [col.lower() for col in df.columns]
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['pulocationid', 'dolocationid']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
df_train = read_dataframe('/workspaces/mlopszoomcamp24/data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('/workspaces/mlopszoomcamp24/data/yellow_tripdata_2023-02.parquet')

In [5]:
df_val.head()

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.00,0.0,1.0,9.40,2.5,0.00,1.683333
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.00,0.0,1.0,74.65,0.0,1.25,32.083333
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.30,0.0,1.0,25.30,2.5,0.00,13.300000
5,1,2023-02-01 00:52:40,2023-02-01 01:07:18,1.0,5.10,1.0,N,148,236,1,21.9,3.50,0.5,5.35,0.0,1.0,32.25,2.5,0.00,14.633333
6,1,2023-02-01 00:12:39,2023-02-01 00:40:36,1.0,8.90,1.0,N,137,244,1,41.5,3.50,0.5,3.50,0.0,1.0,50.00,2.5,0.00,27.950000


In [6]:
len(df_train), len(df_val)

(3009173, 2855951)

In [7]:
df_train.columns

Index(['vendorid', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'ratecodeid', 'store_and_fwd_flag',
       'pulocationid', 'dolocationid', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'duration'],
      dtype='object')

In [8]:
#categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
#categorical = ['pulocationid', 'dolocationid']
#numerical = ['trip_distance']

dv = DictVectorizer()

#train_dicts = df_train[categorical + numerical].to_dict(orient='records')
train_dicts = df_train[['pulocationid', 'dolocationid']].astype(str).to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[['pulocationid', 'dolocationid']].astype(str).to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [10]:
lr = LinearRegression()
lr.fit(X_train, y_train)


# Predict on the training data
y_train_pred = lr.predict(X_train)

# Calculate RMSE on the training data
rmse_train = mean_squared_error(y_train, y_train_pred, squared=False)
print("RMSE on training data:", rmse_train)


# Calculate RMSE on the validation data
y_pred = lr.predict(X_val)

rmse_val = mean_squared_error(y_val, y_pred, squared=False)
print("RMSE on validation data:", rmse_val)

RMSE on training data: 7.649261932106969
RMSE on validation data: 7.811818743246608


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
